In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import os

os.chdir('C:\Working\USA\datafiles')

def write_txt(filename, filetext):
# Write label to txt file
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

In [2]:
# Retrieve data
df = web.DataReader(['^DJI', '^GSPC', '^IXIC'], 'yahoo', 
                    start=datetime(2005, 1, 1)).filter(
                    items=['Adj Close'])[u'Adj Close']

c_names = ['djia', 'sp', 'nsdq']
df.columns = c_names
df.loc[:, 'djia'] = df['djia'].divide(1000)
for col in c_names:
    df.loc[:,col+'_ma'] = df[col].rolling(253).mean()
df = df.dropna()
for col in c_names:
    df.loc[:,col+'_ix'] = df[col].divide(df[col].ix[0] / 100)
df.to_csv('stocks.csv')

# Generate label text
d = {}
for col in c_names:
    for t, v in {'day': -2, 'wk': -6}.iteritems():
        d[col+'_ch'+t] = round((df[col][-1] - df[col][v]) / df[col][v] * 100, 2)
        if d[col+'_ch'+t] > 0:
            d[col+'_ar'+t] = '\color{green!80!black} $ \\blacktriangle $ \\normalcolor'
        else:
            d[col+'_ar'+t] = '\color{red} $\\blacktriangledown$ \\normalcolor'
    d[col+'_mad'] = round((df[col][-1] - df[col+'_ma'][-1]) / df[col+'_ma'][-1] * 100, 2)
    if d[col+'_mad'] > 0:
        d[col+'_mat'] = 'above moving average'
    else:
        d[col+'_mat'] = 'below moving average'
df.loc[:, 'djia'] = df['djia'].multiply(1000)
d1 = df.index[-1].strftime('%b %#d, %Y')
for col in c_names:    
    d[col+'_lt'] = round(df[col][-1], 2)
    d[col+'_t'] = 'One-day change: {} {}\%; \\\ One-week change: {} {}\%'.format(
            d[col+'_arday'], d[col+'_chday'], d[col+'_arwk'], d[col+'_chwk'])
    d[col+'_t2'] = '{}: {:,}; {}\% {}'.format(d1, d[col+'_lt'], d[col+'_mad'], d[col+'_mat'])
    write_txt(col+'_ch.txt', d[col+'_t'])
    write_txt(col+'.txt', d[col+'_t2'])

C:\Users\BDew\Anaconda2\lib\site-packages\pandas_datareader\base.py:177: SymbolWarning: Failed to read symbol: '^DJI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\BDew\Anaconda2\lib\site-packages\pandas_datareader\base.py:177: SymbolWarning: Failed to read symbol: '^GSPC', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\BDew\Anaconda2\lib\site-packages\pandas_datareader\base.py:177: SymbolWarning: Failed to read symbol: '^IXIC', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


RemoteDataError: No data fetched using 'YahooDailyReader'

In [ ]:
df.tail(1)